In [27]:
 %cd /sci/labs/yotamd/lab_share/avishai.wizel/eRNA/

/sci/labs/yotamd/lab_share/avishai.wizel/eRNA


/sci/labs/yotamd/lab_share/avishai.wizel/python_envs/miniconda/envs/tf_gpu_env/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [28]:
import requests
import os
import h5py
import numpy as np
import pandas as pd
from scipy.sparse import csc_matrix
import os 

In [54]:
url = "https://cf.10xgenomics.com/samples/cell-arc/1.0.0/pbmc_granulocyte_sorted_10k/pbmc_granulocyte_sorted_10k_filtered_feature_bc_matrix.h5"
file_path = "./10X_PBMC/01_raw_data/pbmc_granulocyte_sorted_10k_filtered_feature_bc_matrix.h5"

In [55]:
if not os.path.exists("./10X_PBMC/01_raw_data/pbmc_granulocyte_sorted_10k_filtered_feature_bc_matrix.h5"):
    with requests.get(url, stream=True) as r:
            r.raise_for_status() 
            with open(file_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)


In [56]:
with h5py.File(file_path, "r") as f:
    print(f"Keys at root level: {list(f.keys())}") # Show keys at the top level

    # Get the 'matrix' group object
    matrix_group = f['matrix']

    # Now, print the keys *inside* the 'matrix' group
    print(f"Keys inside 'matrix' group: {list(matrix_group.keys())}")

    # You can now access datasets within the 'matrix' group.
    # Common keys for sparse matrices in HDF5 (e.g., from 10x Genomics) are:
    # 'data', 'indices', 'indptr', 'shape'

    # Example: Accessing a dataset named 'data' within the 'matrix' group
    if 'data' in matrix_group:
        data_values = matrix_group['data'][:] # Read the 'data' dataset
        print(f"\nFirst 10 values from 'matrix/data': {data_values[:10]}")
        print(f"Shape of 'matrix/data' dataset: {data_values.shape}")
    else:
        print("\n'data' dataset not found within 'matrix' group.")

    # You can similarly access other datasets like 'indices', 'indptr', 'shape'
    if 'shape' in matrix_group:
        matrix_shape = matrix_group['shape'][:]
        print(f"Shape of the sparse matrix: {matrix_shape}")

    # If you want to list all items recursively (datasets and groups):
    print("\nAll items in the HDF5 file (recursive):")
    f.visititems(lambda name, obj: print(f"{name} ({'Group' if isinstance(obj, h5py.Group) else 'Dataset'})"))

Keys at root level: ['matrix']
Keys inside 'matrix' group: ['barcodes', 'data', 'features', 'indices', 'indptr', 'shape']

First 10 values from 'matrix/data': [1 1 1 1 1 1 1 1 1 6]
Shape of 'matrix/data' dataset: (109104829,)
Shape of the sparse matrix: [144978  11909]

All items in the HDF5 file (recursive):
matrix (Group)
matrix/barcodes (Dataset)
matrix/data (Dataset)
matrix/features (Group)
matrix/features/_all_tag_keys (Dataset)
matrix/features/feature_type (Dataset)
matrix/features/genome (Dataset)
matrix/features/id (Dataset)
matrix/features/interval (Dataset)
matrix/features/name (Dataset)
matrix/indices (Dataset)
matrix/indptr (Dataset)
matrix/shape (Dataset)


In [64]:
import h5py
from scipy.sparse import csr_matrix # או csc_matrix
import numpy as np


with h5py.File(file_path, "r") as f:
    # Load the components of the sparse matrix
    data = f['matrix/data'][:]
    indices = f['matrix/indices'][:]
    indptr = f['matrix/indptr'][:]
    shape = f['matrix/shape'][:] # Returns an array like [rows, cols]

    # Create a sparse matrix (e.g., CSR format)
    # Note: Cell Ranger often outputs in CSC format for ATAC/Gene Exp,
    # so you might need to use csc_matrix if CSR doesn't yield expected results.
    # The 'shape' array from HDF5 usually represents (n_rows, n_cols)
    sparse_matrix = csc_matrix((data, indices, indptr), shape=shape)

    print(f"\nSparse Matrix created with shape: {sparse_matrix.shape}")
    print(f"Non-zero elements: {sparse_matrix.nnz}")
    # You can convert to a dense array (use with caution for very large matrices)
    # dense_matrix = sparse_matrix.toarray()
    # print(f"Dense matrix (first 5x5):\n{dense_matrix[:5, :5]}")
    
    # Access the 'barcodes' dataset
    barcodes_values_bytes  = f['matrix/barcodes'][:]
    barcodes_values = [x.decode('utf-8') for x in barcodes_values_bytes] #decode bytes
    print(f"Data from 'matrix/barcodes' (first 5 values): {barcodes_values[:5]}")
    print(f"Length of 'matrix/barcodes': {len(barcodes_values)}\n")
    
    feature_type = f['matrix/features/feature_type'][:]
    # You can also access datasets within nested groups, like 'matrix/features/id'
    features_id_values_bytes = f['matrix/features/id'][:]
    features_id_values = [x.decode('utf-8') for x in features_id_values_bytes]
    print(f"Data from 'matrix/features/id' (first 5 values): {features_id_values[:5]}")
    print(f"Length of 'matrix/features/id': {len(features_id_values)}\n")



Sparse Matrix created with shape: (144978, 11909)
Non-zero elements: 109104829
Data from 'matrix/barcodes' (first 5 values): ['AAACAGCCAAGGAATC-1', 'AAACAGCCAATCCCTT-1', 'AAACAGCCAATGCGCT-1', 'AAACAGCCACACTAAT-1', 'AAACAGCCACCAACCG-1']
Length of 'matrix/barcodes': 11909

Data from 'matrix/features/id' (first 5 values): ['ENSG00000243485', 'ENSG00000237613', 'ENSG00000186092', 'ENSG00000238009', 'ENSG00000239945']
Length of 'matrix/features/id': 144978



In [33]:
sc_rna_data = pd.DataFrame(
    data=sparse_matrix.toarray().T,
    index= barcodes_values,
    columns= features_id_values 
    )
sc_rna_data

,ENSG00000243485,ENSG00000237613,ENSG00000186092,ENSG00000238009,ENSG00000239945,ENSG00000239906,ENSG00000241860,ENSG00000241599,ENSG00000286448,ENSG00000236601,...,KI270734.1:133761-134101,KI270721.1:2341-2736,KI270726.1:27352-27794,KI270726.1:41529-42186,KI270713.1:4147-4624,KI270713.1:20444-22615,KI270713.1:27118-28927,KI270713.1:29485-30706,KI270713.1:31511-32072,KI270713.1:37129-37638
AAACAGCCAAGGAATC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,8,0,0,0,0
AAACAGCCAATCCCTT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACAGCCAATGCGCT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACAGCCACACTAAT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACAGCCACCAACCG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTGTTAAAC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGGTTAGGATT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGGTTGCAGTA-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGGTTGGTTAG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:

def filter_genes_from_sparse_matrix(
    sparse_matrix,
    gene_names,    # List or array of gene names, matching the order of matrix columns
    genome_assembly="hg38" # "hg38" or "mm10" for human or mouse
):
    """
    Filters a sparse scRNA-seq matrix to remove genes located on sex chromosomes.

    The input matrix is expected to be in a 'cells x genes' (rows x columns) format.

    Args:
        sparse_matrix (scipy.sparse.csr_matrix or scipy.sparse.csc_matrix):
            The sparse matrix containing scRNA-seq data.
            It should be in 'cells x genes' format.
        gene_names (list or np.array):
            A list or NumPy array of gene names, in the same order as the matrix columns.
        genome_assembly (str):
            The genomic reference ("hg38" or "mm10") used for identifying sex chromosomes.

    Returns:
        tuple:
            (scipy.sparse.csr_matrix, pd.Index)
            - The filtered sparse matrix (in CSR format).
            - A pandas.Index containing the names of the retained genes.
    """

    # --- Initial Checks and Information ---
    if not issparse(sparse_matrix):
        raise TypeError("Input 'sparse_matrix' must be a scipy sparse matrix.")
    
    if sparse_matrix.shape[1] != len(gene_names):
        raise ValueError("Number of columns in sparse_matrix must match length of gene_names.")

    # Convert the matrix to CSR format for efficient column slicing (though CSC might be slightly better for pure column ops,
    # CSR is often preferred for overall compatibility with single-cell workflows).
    if not isinstance(sparse_matrix, csr_matrix):
        sparse_matrix = sparse_matrix.tocsr()

    print(f"Initial sparse matrix shape (cells x genes): {sparse_matrix.shape}")
    print(f"Initial number of genes: {sparse_matrix.shape[1]}\n")

    # --- Filtering Genes (Step 1) ---

    print("1. Removing genes encoded on sex chromosomes...")
    
    # Define regular expression patterns for sex chromosomes.
    sex_chroms_patterns = {
        "hg38": r"(?i).*[_-]?(chr)?[XY]$|(?i)^(chr)?[XY][_-].*",
        "mm10": r"(?i).*[_-]?(chr)?[XY]$|(?i)^(chr)?[XY][_-].*"
    }
    
    current_pattern = sex_chroms_patterns.get(genome_assembly.lower())

    if current_pattern is None:
        warnings.warn(f"Unsupported genome assembly '{genome_assembly}'. Skipping sex chromosome gene filtering.")
        # If assembly is not supported, create a mask that retains all genes.
        genes_to_remove_mask = np.zeros(len(gene_names), dtype=bool)
    else:
        # Create a boolean mask: True for genes on sex chromosomes, False otherwise.
        genes_to_remove_mask = pd.Series(gene_names).str.contains(current_pattern, regex=True).to_numpy()

    # Filter the columns of the sparse matrix. The '~' negates the mask.
    final_sparse_matrix = sparse_matrix[:, ~genes_to_remove_mask]
    final_gene_names = pd.Index(np.array(gene_names)[~genes_to_remove_mask])

    print(f"   Genes remaining after sex chromosome removal: {final_sparse_matrix.shape[1]}\n")

    print(f"Final sparse matrix shape (cells x genes): {final_sparse_matrix.shape}")
    print(f"Final number of genes: {final_sparse_matrix.shape[1]}\n")

    return final_sparse_matrix, final_gene_names

In [35]:

sparse_matrix = csr_matrix(sc_rna_data.values)

cell_barcodes = sc_rna_data.index.to_numpy()
gene_names = sc_rna_data.columns.to_numpy()

del(sc_rna_data)


In [37]:
 filtered_sparse_matrix, final_gene_names = filter_genes_from_sparse_matrix(
    sparse_matrix=sparse_matrix,
    gene_names=gene_names
)

Initial sparse matrix shape (cells x genes): (11909, 144978)
Initial number of genes: 144978

1. Removing genes encoded on sex chromosomes...


/tmp/ipykernel_2818915/2837747985.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  genes_to_remove_mask = pd.Series(gene_names).str.contains(current_pattern, regex=True).to_numpy()


   Genes remaining after sex chromosome removal: 144978

Final sparse matrix shape (cells x genes): (11909, 144978)
Final number of genes: 144978



In [41]:
import anndata as ad

scrna_final_adata = ad.AnnData(
    X=filtered_sparse_matrix,        
    obs=pd.DataFrame(index=cell_barcodes),    
    var=pd.DataFrame(index=final_gene_names)     
)



In [43]:
scrna_final_adata.write('./10X_PBMC/03_filtered_data/sc_rna_data.h5ad')


In [61]:
def filter_rna_anndata(
    adata_rna: ad.AnnData,
    genome_assembly: str = "hg38", # "hg38" or "mm10"
    min_genes_per_cell: int = 200
) -> ad.AnnData:
    """
    Performs specific filtering of an AnnData object containing scRNA-seq data:
    1. Removes genes encoded on sex chromosomes.
    2. Removes cells expressing fewer than 'min_genes_per_cell' genes.

    The input AnnData object is expected to have cells as rows (obs) and genes as columns (var).

    Args:
        adata_rna (anndata.AnnData):
            The AnnData object for RNA-seq data (cells x genes).
        genome_assembly (str):
            Genome reference name ("hg38" or "mm10") for sex chromosome identification.
            Assumes gene names in adata.var_names can be parsed for chromosome information
            (e.g., if they contain 'chrX', 'chrY', or similar). If not, this step
            might require 'var' to have a 'chromosome' column.
        min_genes_per_cell (int):
            Minimum number of genes a cell must express to be retained.

    Returns:
        anndata.AnnData: An updated AnnData object containing the filtered matrix and metadata.
                         The matrix (adata.X) will remain in 'cells x genes' format.
    """

    if not isinstance(adata_rna, ad.AnnData):
        raise TypeError("Input 'adata_rna' must be an anndata.AnnData object.")
    
    # Ensure the matrix is sparse, typically CSR for cells x genes
    if not issparse(adata_rna.X):
        warnings.warn("adata_rna.X is not sparse. Converting to CSR sparse matrix.")
        adata_rna.X = csr_matrix(adata_rna.X)

    print(f"Initial AnnData shape (cells x genes): {adata_rna.shape}")
    print(f"Initial number of cells: {adata_rna.shape[0]}")
    print(f"Initial number of genes: {adata_rna.shape[1]}\n")

    # --- 1. Remove genes encoded on sex chromosomes ---
    print("1. Removing genes on sex chromosomes...")
    
    sex_chroms_map = {'hg38': ['chrX', 'chrY'], 'mm10': ['chrX', 'chrY']}
    
    if genome_assembly.lower() not in sex_chroms_map:
        warnings.warn(f"Unsupported genome assembly '{genome_assembly}'. Skipping sex chromosome filtering.")
        is_sex_chrom_gene = np.zeros(adata_rna.shape[1], dtype=bool)
    else:
        current_sex_chroms = sex_chroms_map[genome_assembly.lower()]
        
        # This part assumes gene names might contain chromosome info (e.g., 'chrX_GENE')
        # or that 'adata_rna.var' has a 'chromosome' column.
        # For typical 10x output, gene names are just gene symbols (e.g., 'XIST').
        # A more robust approach might be to map gene symbols to chromosomes using an external annotation.
        # For simplicity, assuming 'adata.var' might have a 'chromosome' column if available from the AnnData creation.
        
        # If 'chromosome' is a column in adata.var, use it:
        if 'chromosome' in adata_rna.var.columns:
            is_sex_chrom_gene = adata_rna.var['chromosome'].isin(current_sex_chroms).values
            print("   Using 'chromosome' column from adata.var for sex chromosome filtering.")
        else:
            # Fallback: try to guess from gene names, or warn.
            # For 10x data, standard gene symbols often don't contain 'chrX' directly.
            # You might need a mapping for specific genes like 'XIST', 'TSIX', 'RPS4Y1', etc.
            # This example attempts to match 'chrX' or 'chrY' directly in gene names.
            # If your gene names are just symbols (e.g., 'XIST', 'SRY'), this might not work
            # and you'd need to provide a list of sex-linked genes.
            is_sex_chrom_gene = np.array([
                any(chrom_prefix in gene_name for chrom_prefix in current_sex_chroms)
                for gene_name in adata_rna.var_names
            ])
            # Add common sex-linked genes manually if not covered by prefix (example for hg38)
            if genome_assembly.lower() == "hg38":
                common_human_sex_genes = ['XIST', 'TSIX', 'SRY', 'RPS4Y1', 'DDX3Y', 'KDM5C', 'EIF1AY']
                is_sex_chrom_gene = is_sex_chrom_gene | adata_rna.var_names.isin(common_human_sex_genes).values
            elif genome_assembly.lower() == "mm10":
                common_mouse_sex_genes = ['Xist', 'Tsix', 'Sry', 'Ddx3y', 'Kdm5c', 'Eif2s3y']
                is_sex_chrom_gene = is_sex_chrom_gene | adata_rna.var_names.isin(common_mouse_sex_genes).values
            
            if not is_sex_chrom_gene.any():
                warnings.warn("   No 'chromosome' column found in adata.var, and no sex chromosome genes identified by name. Sex chromosome filtering might be ineffective.")

    non_sex_chrom_mask = ~is_sex_chrom_gene
    adata_rna = adata_rna[:, non_sex_chrom_mask].copy() # Apply filter to var (columns)

    print(f"   Genes remaining after sex chromosome removal: {adata_rna.shape[1]}\n")

    # --- 2. Remove cells expressing fewer than 'min_genes_per_cell' genes ---
    print(f"2. Removing cells expressing fewer than {min_genes_per_cell} genes...")
    
    # Calculate number of genes expressed per cell for the *current* matrix
    # (adata_rna.X > 0).sum(axis=1) counts non-zero entries per row (cells)
    adata_rna.obs['n_genes_after_sex_chrom_filter'] = (adata_rna.X > 0).sum(axis=1).A1

    cell_filter_mask = (adata_rna.obs['n_genes_after_sex_chrom_filter'] >= min_genes_per_cell)
    
    initial_cells = adata_rna.shape[0]
    adata_rna = adata_rna[cell_filter_mask, :].copy()
    print(f"   Cells remaining after minimum gene expression filter: {adata_rna.shape[0]} (removed {initial_cells - adata_rna.shape[0]} cells)\n")

    print(f"Final AnnData shape (cells x genes): {adata_rna.shape}")
    print(f"Final number of cells: {adata_rna.shape[0]}")
    print(f"Final number of genes: {adata_rna.shape[1]}\n")

    return adata_rna


In [62]:
filtered_rna_adata = filter_rna_anndata(
    adata_rna=adata_rna,
    genome_assembly="hg38", 
    min_genes_per_cell=200
)

NameError: name 'adata_rna' is not defined